## 인스타그램 크롤링
- 인스타그램 게시물을 크롤링하여 벡터DB에 저장할 데이터를 구축한다.

In [24]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import os
import time
import random

In [25]:
instagramAccountName = "ugly_lovely_official"

In [26]:
def random_sleep(min_sleep = 0.01, max_sleep = 0.2):
    random_sec = random.uniform(min_sleep, max_sleep)
    return random_sec

random_sleep(1,2)

1.5988991784173412

In [27]:
# load .env
load_dotenv()

INSTAGRAM_ID = os.environ.get('INSTAGRAM_ID')
INSTAGRAM_PASSWORD = os.environ.get('INSTAGRAM_PASSWORD')

In [28]:
def click_element_by_xpath(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        element.click()
    except Exception as e:
        print(f"Error clicking element by XPATH {xpath}: {e}")

def click_element_by_id(driver, element_id):
    try:
        element = driver.find_element(By.ID, element_id)
        element.click()
    except Exception as e:
        print(f"Error clicking element by ID {element_id}: {e}")

def click_element_by_class_name(driver, class_name):
    try:
        element = driver.find_element(By.CLASS_NAME, class_name)
        element.click()
    except Exception as e:
        print(f"Error clicking element by class name {class_name}: {e}")

def click_element_by_LINK_TEXT(driver, link_text):
    try:
        element = driver.find_element(By.LINK_TEXT, link_text)
        element.click()
    except Exception as e:
        print(f"Error clicking element by class name {link_text}: {e}")



In [40]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# 드라이버 생성
# 크롬 드라이버 서비스 생성
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

wait = WebDriverWait(driver, 5)
driver.implicitly_wait(time_to_wait=10)
driver.get("https://instagram.com")
driver.implicitly_wait(time_to_wait=10)


login_id = driver.find_element(By.CSS_SELECTOR, 'input[name="username"]')
login_id.send_keys(INSTAGRAM_ID) # 아이디 입력
random_sleep()
login_pwd = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
login_pwd.send_keys(INSTAGRAM_PASSWORD) # 비번 입력
random_sleep()

login_id.send_keys(Keys.ENTER) #enter 키를 쳐주세요
driver.implicitly_wait(10)


In [30]:
def add_address(driver,elements_dict) :
    # BeautifulSoup를 사용하여 페이지 소스 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 모든 게시글 링크 추출
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('/p/'):
            element = "https://instagram.com" + a['href']
            elements_dict[f'{element}'] = element
        if a['href'].startswith('/reel/'):
            element = "https://instagram.com" + a['href']
            elements_dict[f'{element}'] = element


In [31]:
adURL= f"https://instagram.com/{instagramAccountName}"
driver.get(adURL)

# 추가 게시물 확인 및 데이터 로딩
# 페이지 로딩 후 스크롤
SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script("return document.body.scrollHeight")
elements_dict = {}

add_address(driver,elements_dict)

while True:
    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 로딩 대기
    time.sleep(SCROLL_PAUSE_TIME)

    add_address(driver,elements_dict)
    print(len(elements_dict))


    # 새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 높이가 변하지 않으면 스크롤 종료
    if new_height == last_height:
        break

    last_height = new_height

12
24
36
48
60
72
83
83


In [32]:
# test
iter_items = iter(elements_dict.items())
next(iter_items)
second_key, second_value = next(iter_items)
driver.get(elements_dict[second_value])

print(len(elements_dict))

83


In [33]:
def get_image_count_in_posts(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    elements_num = len(soup.select('div._acnb'))
    return elements_num

In [34]:
from datetime import datetime

def convert_timestamp(timestamp_str):
    # 입력된 timestamp_str을 datetime 객체로 파싱
    dt = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    
    # 원하는 포맷으로 변환하여 반환
    return dt.strftime('%Y.%m.%d %H:%M:%S')

In [35]:
import requests
import os
import mimetypes

def download_image_from_url(image_src, index, save_dir='./images'):
    save_as = os.path.join(save_dir, f"{index}.jpg")
    
    # 이미지 다운로드
    response = requests.get(image_src)
    if response.status_code == 200:
        with open(save_as, "wb") as f:
            f.write(response.content)
        
        # Content-Type 헤더를 이용하여 파일 확장자 추출
        content_type = response.headers.get('content-type')
        ext = mimetypes.guess_extension(content_type)
        if ext:
            # 다운로드한 파일의 확장자를 변경
            new_file_name = save_as.replace(".jpg", ext)
            os.rename(save_as, new_file_name)
            save_as = new_file_name
        
        print(f"이미지 다운로드 완료: {save_as}")
        return save_as
    else:
        print(f"이미지 다운로드 실패: HTTP 상태 코드 {response.status_code}")
        return False
    

In [36]:
def get_post_text(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract the time element
    time_element = soup.find('time', class_='xsgj6o6')
    datetime = convert_timestamp(time_element['datetime'])

    # Extract the content
    content_element = soup.find('span', class_='x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs xt0psk2 x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj')
    
    # Extract the hashtags
    hashtags = [a.get_text() for a in content_element.find_all('a')]
    
    # Remove <a> tags from the content
    for a in content_element.find_all('a'):
        a.decompose()

    content = ''.join(content_element.stripped_strings)


    # Construct the JSON data
    data = {
        '생성시간': datetime,
        '게시글내용': content,
        '해쉬태그': hashtags
    }
    return data

In [37]:
import csv

def dict_to_csv(data, csv_filename='output.csv'):
    # 딕셔너리의 키 값을 CSV 파일의 헤더로 사용
    fields = list(data[0].keys())  # Assuming all dictionaries in data have the same keys
    
    # CSV 파일 쓰기 모드로 열기
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['Index'] + fields)
        writer.writeheader()
        
        # 각 데이터를 순회하며 CSV 파일에 쓰기
        for idx, entry in enumerate(data, start=1):
            row = {'Index': idx}
            row.update(entry)
            writer.writerow(row)
    
    print(f'CSV 파일 "{csv_filename}" 저장 완료.')

In [42]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 각 게시글의 텍스트와 이미지를 추출
data = []
xpath = "//img[@class='x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3']"
for index, value in enumerate(list(elements_dict)[:5]):
    
    driver.get(value)
    
    num=get_image_count_in_posts(driver)
    print(num)
    
    # 게시글 모든 사진 가져오기
    # for i in range(1,num+1):
    #     driver.get(f"{value}?img_index={i}")

    image_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
        )
    
    image_src = image_element.get_attribute("src")
    label = str(index)
    save_img = download_image_from_url(image_src, label)

    post_text = get_post_text(driver)

    post_text['img_path']= save_img
    data.append(post_text)
    
dict_to_csv(data)

0
이미지 다운로드 완료: ./images\0.jpg
0
이미지 다운로드 완료: ./images\1.jpg
2
이미지 다운로드 완료: ./images\2.jpg
0
이미지 다운로드 완료: ./images\3.jpg
0
이미지 다운로드 완료: ./images\4.jpg
CSV 파일 "output.csv" 저장 완료.
